In [ ]:
# set-up colab
from google.colab import drive
from tqdm import tqdm
drive.mount("/content/drive")
%cd '/content/drive/My Drive/Dissertation'
import pandas as pd
import ast
import pickle

# reading the files 
with open("observable_similarity_list", "rb") as fp1: 
  visible_similarity_matrix_list = pickle.load(fp1) 

with open("unobservable_similarity_list", "rb") as fp2: 
  non_visible_similarity_matrix_list = pickle.load(fp2) 

with open("recommendation_id", "rb") as fp3: 
  recommendation_id_list = pickle.load(fp3)

# expanding the dataset vertically so we can perform our analysis in R
def extending_out(id_list,original_list_of_list):
  new_list,new_id_list,order,=[],[],[]
  for i in tqdm(range(len(original_list_of_list))):
    current=original_list_of_list[i]
    id=str(id_list[i])
    for j in range(len(current)):
      t=int(j+1)
      score=float(current[j])
      new_list.append(score)
      new_id_list.append(id)
      order.append(t)
  return(order,new_id_list,new_list)

# executing the verticle expansion for the default observed comments 
order,new_recommendation_ids,observable_comments_list=extending_out(recommendation_id_list,visible_similarity_matrix_list)
typeof=["observable" for i in range(len(new_recommendation_ids))]
df1 = pd.DataFrame(list(zip(new_recommendation_ids,observable_comments_list,order,typeof)), columns=['recommendation_id','similarity_score','order','type'])  

# executing the verticle expansion for the side-bar display of recent reviews
order,new_recommendation_ids,unobservable_comments_list=extending_out(recommendation_id_list,non_visible_similarity_matrix_list)
typeof=["unobservable" for i in range(len(new_recommendation_ids))]
df2 = pd.DataFrame(list(zip(new_recommendation_ids,unobservable_comments_list,order,typeof)), columns =['recommendation_id','similarity_score','order','type'])  

# combining the 2 together in one dataframe 
frames = [df1, df2]
data= pd.concat(frames)
data.to_csv('reviewer_analysis.csv')
data

Mounted at /content/drive
/content/drive/My Drive/Dissertation


100%|██████████| 920536/920536 [00:05<00:00, 162167.54it/s]


,recommendation_id,similarity_score,order,type
0,117089901,0.606679,1,observable
1,117089901,0.411201,2,observable
2,117089901,0.534113,3,observable
3,117089901,0.423936,4,observable
4,117089901,0.566998,5,observable
...,...,...,...,...
9205355,100801803,0.802641,6,unobservable
9205356,100801803,0.557524,7,unobservable
9205357,100801803,0.838760,8,unobservable
9205358,100801803,0.721979,9,unobservable
